In [16]:
import pandas as pd
import plotly.graph_objects as go
from evento import ajustar_datas

In [17]:
# Fundo sem histórico completo de preços no Yahoo Finance
# Carregar histórico da B3
cotas = (
    pd.read_csv('./data/knri_tr.csv', parse_dates=['datneg'])
    # a série da B3 começa em "2010-12-01" -> vamos começar em "2011-01-01"
    .query('datneg >= "2011-01-01"')
)
cotas

,datneg,preult_tr
0,2011-01-04,100.000000
1,2011-01-05,99.700000
2,2011-01-06,99.700000
3,2011-01-07,99.500000
4,2011-01-10,95.500000
...,...,...
2973,2023-01-09,324.315603
2974,2023-01-10,324.315603
2975,2023-01-11,325.291687
2976,2023-01-12,323.967001


In [18]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=cotas["datneg"],
        y=cotas["preult_tr"],
        mode="lines",
        name="cotas ajustada (Total Return)",
        line=dict(color="#02878e"),
    )
)
fig.update_layout(
    font=dict(family="Fira Code", size=11, color="black"),
    title="KNRI11 Total Return <br> (reinvestimento dos dividendos na data-ex)",
    title_x=0.5,
    title_y=0.85,
    xaxis_title="Data",
    yaxis_title="Preço (R$)",
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01),
)
fig.show()

In [19]:
# http://estatisticas.cetip.com.br/astec/series_v05/paginas/lum_web_v05_template_informacoes_di.asp?str_Modulo=completo&int_Idioma=1&int_Titulo=6&int_NivelBD=2
di = pd.read_csv("data/taxa_di.csv", sep=",", decimal=".", parse_dates=["data"], dayfirst=True)
di

,data,media,fator_diario,taxa_selic
0,2011-01-04,10.64,1.000401,10.66
1,2011-01-05,10.64,1.000401,10.67
2,2011-01-06,10.64,1.000401,10.66
3,2011-01-07,10.64,1.000401,10.67
4,2011-01-10,10.64,1.000401,10.66
...,...,...,...,...
3018,2023-01-09,13.65,1.000508,13.65
3019,2023-01-10,13.65,1.000508,13.65
3020,2023-01-11,13.65,1.000508,13.65
3021,2023-01-12,13.65,1.000508,13.65


In [20]:
di["di_acum"] = di["fator_diario"].cumprod()
di

,data,media,fator_diario,taxa_selic,di_acum
0,2011-01-04,10.64,1.000401,10.66,1.000401
1,2011-01-05,10.64,1.000401,10.67,1.000803
2,2011-01-06,10.64,1.000401,10.66,1.001204
3,2011-01-07,10.64,1.000401,10.67,1.001606
4,2011-01-10,10.64,1.000401,10.66,1.002008
...,...,...,...,...,...
3018,2023-01-09,13.65,1.000508,13.65,2.802877
3019,2023-01-10,13.65,1.000508,13.65,2.804300
3020,2023-01-11,13.65,1.000508,13.65,2.805724
3021,2023-01-12,13.65,1.000508,13.65,2.807149


In [21]:
di.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3023 entries, 0 to 3022
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   data          3023 non-null   datetime64[ns]
 1   media         3023 non-null   float64       
 2   fator_diario  3023 non-null   float64       
 3   taxa_selic    3023 non-null   float64       
 4   di_acum       3023 non-null   float64       
dtypes: datetime64[ns](1), float64(4)
memory usage: 118.2 KB


In [22]:
cotas = cotas.merge(di[["data", "di_acum"]], how="left", left_on="datneg", right_on="data")
cotas.drop(columns=["data"], inplace=True)
cotas

,datneg,preult_tr,di_acum
0,2011-01-04,100.000000,1.000401
1,2011-01-05,99.700000,1.000803
2,2011-01-06,99.700000,1.001204
3,2011-01-07,99.500000,1.001606
4,2011-01-10,95.500000,1.002008
...,...,...,...
2973,2023-01-09,324.315603,2.802877
2974,2023-01-10,324.315603,2.804300
2975,2023-01-11,325.291687,2.805724
2976,2023-01-12,323.967001,2.807149


In [23]:
cotas["di_acum"] = cotas["preult_tr"].iloc[0] * cotas["di_acum"]
cotas

,datneg,preult_tr,di_acum
0,2011-01-04,100.000000,100.040132
1,2011-01-05,99.700000,100.080280
2,2011-01-06,99.700000,100.120444
3,2011-01-07,99.500000,100.160625
4,2011-01-10,95.500000,100.200821
...,...,...,...
2973,2023-01-09,324.315603,280.287671
2974,2023-01-10,324.315603,280.430023
2975,2023-01-11,325.291687,280.572448
2976,2023-01-12,323.967001,280.714945


In [36]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=cotas["datneg"],
        y=cotas["preult_tr"],
        mode="lines",
        name="Cota TR",
        line=dict(color="#e86f00"),
    )
)
fig.add_trace(
    go.Scatter(
        x=cotas["datneg"],
        y=cotas["di_acum"],
        mode="lines",
        name="DI Acumulado",
        line=dict(color="#02878e"),
    )
)
fig.update_layout(
    font=dict(family="Fira Code", size=11, color="black"),
    title="Cotas Total Return (TR) do KNRI11",
    title_x=0.5,
    title_y=0.85,
    xaxis_title="Data",
    yaxis_title="Cota TR (R$)",
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01),
)
fig.show()

In [25]:
# https://www.ibge.gov.br/estatisticas/economicas/precos-e-custos/9256-indice-nacional-de-precos-ao-consumidor-amplo.html?t=downloads&utm_source=landing&utm_medium=explica&utm_campaign=inflacao#plano-real-mes
ipca = (pd
    .read_csv('data/ipca.csv', parse_dates=['mes'])
    .query('mes >= "2011-01-01"')
    .rename(columns={'mes':'datneg'})
    .reset_index(drop=True)
)
ipca

,datneg,indice
0,2011-01-01,3222.42
1,2011-02-01,3248.20
2,2011-03-01,3273.86
3,2011-04-01,3299.07
4,2011-05-01,3314.58
...,...,...
139,2022-08-01,6388.87
140,2022-09-01,6370.34
141,2022-10-01,6407.93
142,2022-11-01,6434.20


In [26]:
ipca.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144 entries, 0 to 143
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   datneg  144 non-null    datetime64[ns]
 1   indice  144 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 2.4 KB


In [27]:
# Normalizar o início da série de IPCA para 100
ipca['indice_ipca'] = ipca['indice'] / ipca['indice'].iloc[0]
# Coluna indice não é mais necessária
ipca.drop(columns='indice', inplace=True)
ipca

,datneg,indice_ipca
0,2011-01-01,1.000000
1,2011-02-01,1.008000
2,2011-03-01,1.015963
3,2011-04-01,1.023786
4,2011-05-01,1.028600
...,...,...
139,2022-08-01,1.982631
140,2022-09-01,1.976881
141,2022-10-01,1.988546
142,2022-11-01,1.996698


In [28]:
cotas = cotas.merge(ipca, how='outer')
cotas.sort_values('datneg', ignore_index=True, inplace=True)
cotas['indice_ipca'].fillna(method='ffill', inplace=True)
# A operação de merge adicionará linhas com NaN em "preult_tr"
# nos dias de índice sem cotação -> vamos removê-las
cotas.dropna(subset=['preult_tr'], inplace=True)
# Calcular a cotação ajustada para o IPCA
cotas['preult_tr_real'] = cotas['preult_tr'] / cotas['indice_ipca']
cotas

,datneg,preult_tr,di_acum,indice_ipca,preult_tr_real
1,2011-01-04,100.000000,100.040132,1.000000,100.000000
2,2011-01-05,99.700000,100.080280,1.000000,99.700000
3,2011-01-06,99.700000,100.120444,1.000000,99.700000
4,2011-01-07,99.500000,100.160625,1.000000,99.500000
5,2011-01-10,95.500000,100.200821,1.000000,95.500000
...,...,...,...,...,...
3031,2023-01-09,324.315603,280.287671,2.009077,161.425171
3032,2023-01-10,324.315603,280.430023,2.009077,161.425171
3033,2023-01-11,325.291687,280.572448,2.009077,161.911008
3034,2023-01-12,323.967001,280.714945,2.009077,161.251658


In [37]:
# Plotar o gráfico
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=cotas['datneg'],
        y=cotas['preult_tr_real'],
        mode='lines',
        name='Cota Real Total Return',
        line=dict(color='#e86f00'),
    )
)
fig.update_layout(
    font=dict(family='Fira Code', size=11, color='black'),
    title='Cotas Real Total Return (RTR) do KNRI11 <br> (IPCA inflation-adjusted total return)',
    title_x=0.5,
    title_y=0.85,
    xaxis_title='Data',
    yaxis_title='Cota RTR (R$)',
    legend=dict(yanchor='top', y=0.99, xanchor='left', x=0.01),
)
fig.show()

In [42]:
# Ganho real do fundo
ganho_real_periodo = (cotas['preult_tr_real'].iloc[-1] - cotas['preult_tr_real'].iloc[0]) / cotas['preult_tr_real'].iloc[0]
print(f'Ganho real no período  = {ganho_real_periodo:.2%}')
years = (cotas['datneg'].iloc[-1] - cotas['datneg'].iloc[-0]).days / 365
print(f'Total de anos no período = {years:.2f} anos')
# Ganho real anualizado
ganho_real_periodo_anualizado = (ganho_real_periodo + 1) ** (1 / years) - 1
print(f'Ganho real anualizado = {ganho_real_periodo_anualizado:.2%}')

Ganhor real no período  = 60.56%
Total de anos no periodo = 12.03 anos
Ganho real anualizado = 4.01%


In [31]:
# https://data.anbima.com.br/indices/?utm_source=anbima.com.br/pt_br&utm_medium=banner_indices&utm_campaign=banner_indices_portal&_ga=2.43112549.1151568226.1673173175-2062883459.1673173174
# Dados para o IMAB5
imab5 = (pd
    .read_csv('data/imab5.csv', parse_dates=['data'], dayfirst=True)
    .query('data >= "2011-01-01"')
    .rename(columns={'data':'datneg'})
    .reset_index(drop=True)
)
imab5

,datneg,imab5
0,2011-01-03,2.230022
1,2011-01-04,2.230661
2,2011-01-05,2.231823
3,2011-01-06,2.232618
4,2011-01-07,2.231839
...,...,...
3020,2023-01-11,8.051167
3021,2023-01-12,8.063897
3022,2023-01-13,8.066159
3023,2023-01-16,8.058740


In [32]:
# Normalizar o início do índice para 100
imab5['imab5_norm'] = 100* imab5['imab5'] / imab5['imab5'].iloc[0]
# Coluna indice não é mais necessária
imab5.drop(columns='imab5', inplace=True)
imab5

,datneg,imab5_norm
0,2011-01-03,100.000000
1,2011-01-04,100.028639
2,2011-01-05,100.080761
3,2011-01-06,100.116378
4,2011-01-07,100.081453
...,...,...
3020,2023-01-11,361.035245
3021,2023-01-12,361.606091
3022,2023-01-13,361.707539
3023,2023-01-16,361.374867


In [33]:
# Juntar os dois índices
cotas = cotas.merge(imab5, how='left')
cotas

,datneg,preult_tr,di_acum,indice_ipca,preult_tr_real,imab5_norm
0,2011-01-04,100.000000,100.040132,1.000000,100.000000,100.028639
1,2011-01-05,99.700000,100.080280,1.000000,99.700000,100.080761
2,2011-01-06,99.700000,100.120444,1.000000,99.700000,100.116378
3,2011-01-07,99.500000,100.160625,1.000000,99.500000,100.081453
4,2011-01-10,95.500000,100.200821,1.000000,95.500000,100.019776
...,...,...,...,...,...,...
2974,2023-01-09,324.315603,280.287671,2.009077,161.425171,358.807859
2975,2023-01-10,324.315603,280.430023,2.009077,161.425171,360.061165
2976,2023-01-11,325.291687,280.572448,2.009077,161.911008,361.035245
2977,2023-01-12,323.967001,280.714945,2.009077,161.251658,361.606091


In [35]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=cotas["datneg"],
        y=cotas["preult_tr"],
        mode="lines",
        name="Cota TR",
        line=dict(color="#e86f00"),
    )
)
fig.add_trace(
    go.Scatter(
        x=cotas["datneg"],
        y=cotas["imab5_norm"],
        mode="lines",
        name="IMA-B 5",
        line=dict(color="#02878e"),
    )
)

fig.update_layout(
    font=dict(family="Fira Code", size=11, color="black"),
    title="Cotas Total Return (TR) <br> (KNRI11 x IMA-B 5)",
    title_x=0.5,
    title_y=0.85,
    xaxis_title="Data",
    yaxis_title="Cota TR (R$)",
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01),
)
fig.show()